<a href="https://colab.research.google.com/github/Emmahgithub/Administraci-n-De-Riesgos-Financieros/blob/main/Riesgo_Historico_Valuacion_En_MXM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESTE ES EL EJERCICIO EXTRA BASICAMENTE ES LO MISMO PERO AGREGA EL HECHO DE QUE SI NO TIENES



In [1]:
# ============================================================
# CUADRO 0: IMPORTS Y PARÁMETROS
# ============================================================

!git clone https://github.com/Emmahgithub/Administraci-n-De-Riesgos-Financieros.git

import numpy as np
import pandas as pd
import yfinance as yf

from datetime import datetime
from math import sqrt, isfinite

# ---- Parámetros generales ----
VAL_DATE_STR = "2025-09-08"
VAL_DATE = pd.Timestamp(VAL_DATE_STR)

DAY_COUNT = 360.0         # base simple 30/360 para tasas
ALPHA = 0.99              # nivel de confianza VaR/ES
LOOKBACK_DAYS = 1000      # ventana histórica base
HORIZON_N_DAYS = 1        # horizonte de riesgo n-días (default 1)
EWMA_LAMBDA = 0.94        # alisado EWMA para pesos temporales

# ---- Insumos locales (ajusta rutas si cambian) ----
PATH_DIRS   = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tasa_DIRS_SW_OP.txt"  # curva MXN (TIIE/DIRS)  %
PATH_LIBOR  = "/content/Administraci-n-De-Riesgos-Financieros/Tasas/tasa_libor.txt"       # curva USD (LIBOR)      %
FX_TICK     = "MXN=X"                          # USD/MXN (MXN por USD)

np.random.seed(42)


Cloning into 'Administraci-n-De-Riesgos-Financieros'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 43 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 184.98 KiB | 5.29 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [2]:
# ============================================================
# CUADRO 1: LECTURA DE CURVAS (robusta)
#  - Archivos con encabezado: DATE, 1, 7, 30, ...
#  - Tasas en % nominal anual
# ============================================================

def _read_curve_txt_ccs(path: str) -> pd.DataFrame:
    df = pd.read_csv(path, sep=r"[,\s;|]+", engine="python")
    first = df.columns[0]
    df = df.rename(columns={first: "DATE"})
    df["DATE"] = pd.to_datetime(df["DATE"], errors="coerce")
    df = df.dropna(subset=["DATE"]).set_index("DATE").sort_index()
    # columnas de tenor a int
    cols = {}
    for c in df.columns:
        try:
            cols[int(str(c).strip())] = pd.to_numeric(df[c], errors="coerce")
        except:
            pass
    if not cols:
        raise ValueError(f"No se hallaron tenores numéricos en {path}")
    out = pd.DataFrame(cols, index=df.index).sort_index(axis=1)
    return out

# Curvas MXN / USD
curve_mxn = _read_curve_txt_ccs(PATH_DIRS)   # proyección y descuento MXN
curve_usd = _read_curve_txt_ccs(PATH_LIBOR)  # proyección y descuento USD


In [3]:
# ============================================================
# CUADRO 2: ALAMBRADA, DF(t) y FORWARD rates
# ============================================================
import numpy as np

def alambrada_rate_at_ccs(row: pd.Series, t_days: int) -> float:
    ten = np.array(row.index, dtype=int)
    y   = np.array(row.values, dtype=float)
    if len(ten) == 0:
        return 0.0
    if t_days <= ten[0]:
        if len(ten) == 1: return float(y[0])
        slope = (y[1]-y[0])/(ten[1]-ten[0])
        return float(y[0] + slope*(t_days-ten[0]))
    if t_days >= ten[-1]:
        if len(ten) == 1: return float(y[-1])
        slope = (y[-1]-y[-2])/(ten[-1]-ten[-2])
        return float(y[-1] + slope*(t_days-ten[-1]))
    i = np.searchsorted(ten, t_days)
    t0,t1 = ten[i-1],ten[i]
    y0,y1 = y[i-1],y[i]
    w = (t_days-t0)/(t1-t0)
    return float(y0*(1-w)+y1*w)

def _pick_row_on_or_before(panel: pd.DataFrame, when: pd.Timestamp) -> pd.Series:
    if when in panel.index:
        return panel.loc[when].dropna()
    pos = panel.index.searchsorted(when)
    pos = max(0, pos-1)
    return panel.iloc[pos].dropna()

def df_func_from_curve(panel: pd.DataFrame, when: pd.Timestamp):
    row = _pick_row_on_or_before(panel, when)
    def df(t_days: int) -> float:
        r_pct = alambrada_rate_at_ccs(row, max(int(t_days),1))
        r = r_pct/100.0
        t = max(t_days,0)/DAY_COUNT
        return 1.0/(1.0 + r*t)     # interés simple base 360
    return df

def forward_rate_from_curve(panel: pd.DataFrame, when: pd.Timestamp, start_d: int, tau_d: int) -> float:
    D = df_func_from_curve(panel, when)
    t0 = max(start_d,0); t1 = t0 + max(tau_d,1)
    D0, D1 = D(t0), D(t1)
    tau = (t1-t0)/DAY_COUNT
    return (D0/D1 - 1.0)/max(tau,1e-9)


In [4]:
# ============================================================
# CUADRO 3: USD/MXN HISTÓRICO Y "HOY"
# ============================================================

def _last_business_before(t: pd.Timestamp, s: pd.Series) -> pd.Timestamp:
    if t in s.index: return t
    pos = s.index.searchsorted(t)
    pos = max(0, pos-1)
    return s.index[pos]

def get_fx_panel(end_date=VAL_DATE, lookback_days=LOOKBACK_DAYS, ticker=FX_TICK):
    start = (end_date - pd.Timedelta(days=lookback_days*2)).strftime("%Y-%m-%d")
    end   = (end_date + pd.Timedelta(days=3)).strftime("%Y-%m-%d")
    raw = yf.download([ticker], start=start, end=end, auto_adjust=True, progress=False)
    if raw is None or len(raw)==0:
        raise RuntimeError("No hay datos de yfinance para USD/MXN.")
    if isinstance(raw.columns, pd.MultiIndex) and "Close" in raw.columns.get_level_values(0):
        fx = raw["Close"].copy()
    elif "Close" in raw.columns:
        fx = raw["Close"].to_frame(name=ticker)
    elif "Adj Close" in raw.columns:
        fx = raw["Adj Close"].to_frame(name=ticker)
    else:
        # fallback
        fx = raw.iloc[:,0].to_frame(name=ticker)
    fx = fx.dropna().ffill()
    last = _last_business_before(end_date, fx[ticker])
    fx = fx.loc[:last].tail(lookback_days+2).ffill().dropna()
    return fx

FX_PANEL = get_fx_panel()
def spot_usdmxn_today() -> float:
    s = FX_PANEL[FX_TICK]
    return float(s.loc[_last_business_before(VAL_DATE, s)])


In [5]:
# ============================================================
# CUADRO 4: PORTAFOLIO CCS (MXN ↔ USD flotante vs flotante)
#  Campos por CCS:
#   - notional_mxn, notional_usd
#   - ttm_days, freq_days
#   - receive_mxn: True si recibes MXN-float y pagas USD-float
#   - exchange_initial, exchange_final: intercambio de nocionales
# ============================================================

CCS_PORT = [
    {
        "name": "CCS_1",
        "notional_mxn": 160_000_000.0,
        "notional_usd": 8_500_000.0,      # aprox. MNX / spot al inicio (solo referencia)
        "ttm_days": 720,                  # ~ 2 años
        "freq_days": 28,                  # paguitos cada 28 días
        "receive_mxn": True,              # recibes MXN, pagas USD
        "exchange_initial": False,        # sin intercambio inicial
        "exchange_final"  : True          # intercambio de nocional al vencimiento
    },
    # Puedes agregar más CCS aquí...
]


In [6]:
# ============================================================
# CUADRO 5: VALUACIÓN "HOY" DEL CCS (PV en MXN)
#  - Proyección MXN con DIRS (curve_mxn) y USD con LIBOR (curve_usd)
#  - Descuento por la misma curva de cada moneda
#  - PV USD se convierte a MXN con spot USD/MXN (MXN=X)
# ============================================================

def pv_ccs_today(ccs: dict, when=VAL_DATE) -> float:
    Nmxn = float(ccs["notional_mxn"])
    Nusd = float(ccs["notional_usd"])
    T    = int(ccs["ttm_days"])
    f    = int(ccs["freq_days"])
    recv_mxn = bool(ccs.get("receive_mxn", True))
    exch0 = bool(ccs.get("exchange_initial", False))
    exchT = bool(ccs.get("exchange_final", True))

    Dmxn = df_func_from_curve(curve_mxn, when)
    Dusd = df_func_from_curve(curve_usd, when)

    # Forward rates por periodo
    Np = max(1, int(np.ceil(T/f)))
    tau = f/DAY_COUNT

    # Pierna MXN (en MXN)
    pv_mxn = 0.0
    for k in range(1, Np+1):
        t0 = (k-1)*f; tk = min(k*f, T)
        fwd_mxn = forward_rate_from_curve(curve_mxn, when, t0, tk-t0)
        cf = Nmxn * fwd_mxn * tau
        pv_mxn += cf * Dmxn(tk)
    if exchT:
        pv_mxn += Nmxn * Dmxn(T)  # devolución de nocional

    # Pierna USD (en USD, luego MXN)
    pv_usd = 0.0
    for k in range(1, Np+1):
        t0 = (k-1)*f; tk = min(k*f, T)
        fwd_usd = forward_rate_from_curve(curve_usd, when, t0, tk-t0)
        cf = Nusd * fwd_usd * tau
        pv_usd += cf * Dusd(tk)
    if exchT:
        pv_usd += Nusd * Dusd(T)

    fx = spot_usdmxn_today()   # MXN por USD
    pv_usd_mxn = pv_usd * fx

    # Sentido del swap
    pv_total = (pv_mxn - pv_usd_mxn) if recv_mxn else (pv_usd_mxn - pv_mxn)
    return float(pv_total)

# Valor hoy del portafolio CCS
PV_TODAY_SER = pd.Series({c["name"]: pv_ccs_today(c) for c in CCS_PORT})
PV_TODAY_TOT = float(PV_TODAY_SER.sum())

print("PV hoy por CCS (MXN):")
display(PV_TODAY_SER.round(2))
print(f"PV total portafolio CCS (MXN) al {VAL_DATE_STR}: {PV_TODAY_TOT:,.2f}")


PV hoy por CCS (MXN):


,0
CCS_1,908832.38


PV total portafolio CCS (MXN) al 2025-09-08: 908,832.38


In [7]:
# ============================================================
# CUADRO 6: ESCENARIOS HISTÓRICOS
#  - Retornos n-días para USD/MXN
#  - Cambios n-días de tasas (Δy) por tenor para DIRS y LIBOR
# ============================================================

def make_returns(series: pd.Series, n_days: int):
    return series/series.shift(n_days) - 1.0

def curve_shift(panel: pd.DataFrame, n_days: int) -> pd.DataFrame:
    return (panel - panel.shift(n_days)).dropna()

# Históricos
rets_fx = make_returns(FX_PANEL[FX_TICK], HORIZON_N_DAYS).dropna()

dMXN = curve_shift(curve_mxn, HORIZON_N_DAYS)
dUSD = curve_shift(curve_usd, HORIZON_N_DAYS)

# Fechas comunes
common_idx = rets_fx.index.intersection(dMXN.index).intersection(dUSD.index)
rets_fx = rets_fx.loc[common_idx]
dMXN = dMXN.loc[common_idx]
dUSD = dUSD.loc[common_idx]

print(f"Escenarios listos (n={HORIZON_N_DAYS} día[s]): {len(common_idx)} fechas.")


Escenarios listos (n=1 día[s]): 155 fechas.


In [8]:
# ============================================================
# CUADRO 7: VALUACIÓN POR ESCENARIO Y P&L
# ============================================================

def df_from_row_ccs(row: pd.Series):
    def df(t_days: int) -> float:
        r_pct = alambrada_rate_at_ccs(row, max(int(t_days),1))
        r = r_pct/100.0
        t = max(t_days,0)/DAY_COUNT
        return 1.0/(1.0 + r*t)
    return df

def forward_from_row_ccs(row: pd.Series, start_d: int, tau_d: int) -> float:
    D = df_from_row_ccs(row)
    t0 = max(start_d,0); t1 = t0 + max(tau_d,1)
    D0, D1 = D(t0), D(t1)
    tau = (t1-t0)/DAY_COUNT
    return (D0/D1 - 1.0)/max(tau,1e-9)

def pv_ccs_in_scenario(ccs: dict, drow_mxn: pd.Series, drow_usd: pd.Series, r_fx: float) -> float:
    Nmxn = float(ccs["notional_mxn"])
    Nusd = float(ccs["notional_usd"])
    T    = int(ccs["ttm_days"])
    f    = int(ccs["freq_days"])
    recv_mxn = bool(ccs.get("receive_mxn", True))
    exchT = bool(ccs.get("exchange_final", True))

    # Curva "hoy" + Δy (aditivo por tenor)
    base_mxn = _pick_row_on_or_before(curve_mxn, VAL_DATE).copy()
    base_usd = _pick_row_on_or_before(curve_usd, VAL_DATE).copy()
    for ten in drow_mxn.index.intersection(base_mxn.index):
        base_mxn.loc[ten] = base_mxn.loc[ten] + drow_mxn.loc[ten]
    for ten in drow_usd.index.intersection(base_usd.index):
        base_usd.loc[ten] = base_usd.loc[ten] + drow_usd.loc[ten]

    Dmxn = df_from_row_ccs(base_mxn)
    Dusd = df_from_row_ccs(base_usd)

    Np = max(1, int(np.ceil(T/f)))
    tau = f/DAY_COUNT

    # Pierna MXN (en MXN)
    pv_mxn = 0.0
    for k in range(1, Np+1):
        t0 = (k-1)*f; tk = min(k*f, T)
        fwd_mxn = forward_from_row_ccs(base_mxn, t0, tk-t0)
        pv_mxn += (Nmxn * fwd_mxn * tau) * Dmxn(tk)
    if exchT:
        pv_mxn += Nmxn * Dmxn(T)

    # Pierna USD (en USD), luego a MXN con FX escenario
    pv_usd = 0.0
    for k in range(1, Np+1):
        t0 = (k-1)*f; tk = min(k*f, T)
        fwd_usd = forward_from_row_ccs(base_usd, t0, tk-t0)
        pv_usd += (Nusd * fwd_usd * tau) * Dusd(tk)
    if exchT:
        pv_usd += Nusd * Dusd(T)

    fx0 = spot_usdmxn_today()
    fx1 = float(fx0 * (1.0 + r_fx))   # aplica retorno n-días
    pv_usd_mxn = pv_usd * fx1

    pv_total = (pv_mxn - pv_usd_mxn) if recv_mxn else (pv_usd_mxn - pv_mxn)
    return float(pv_total)

# Matriz de valuaciones por escenario
vals_scen = []
for dt in common_idx:
    row_m = dMXN.loc[dt]
    row_u = dUSD.loc[dt]
    rfx   = rets_fx.loc[dt]
    scen = {}
    for c in CCS_PORT:
        scen[c["name"]] = pv_ccs_in_scenario(c, row_m, row_u, rfx)
    vals_scen.append(pd.Series(scen, name=dt))

VALS_SCEN = pd.DataFrame(vals_scen).sort_index()
PVAL_SCEN = VALS_SCEN.sum(axis=1)

# P&L
PV0 = PV_TODAY_SER
PNL_SCEN = VALS_SCEN.sub(PV0, axis=1)
PNL_PTF  = PVAL_SCEN - float(PV_TODAY_SER.sum())

print("Listo: matrices por escenario y P&L generadas.")


Listo: matrices por escenario y P&L generadas.


In [9]:
# ============================================================
# CUADRO 8: VaR / ES (histórico puro y con EWMA) + helpers
# ============================================================

def var_cvar_series(losses: pd.Series, alpha=ALPHA, weights=None):
    x = losses.values.astype(float)
    if weights is None:
        q = np.quantile(x, alpha, method="linear")
        cvar = float(x[x >= q].mean())
        return float(q), float(cvar)
    w = np.asarray(weights, float); w = w/w.sum()
    idx = np.argsort(x)
    xs, ws = x[idx], w[idx]
    cws = np.cumsum(ws)
    j = np.searchsorted(cws, alpha)
    q = float(xs[min(j, len(xs)-1)])
    mask = xs >= q
    wt = ws[mask]; xt = xs[mask]
    cvar = float((wt*xt).sum()/max(wt.sum(),1e-12)) if wt.sum()>0 else q
    return float(q), float(cvar)

def ewma_weights(n: int, lam=EWMA_LAMBDA):
    w = np.array([lam**(n-1-i) for i in range(n)], float)
    return w/w.sum()

# Pérdidas (definición: pérdida positiva = -PNL si PNL<0)
LOSS_PTF = -PNL_PTF
LOSS_BY  = -PNL_SCEN

# Portafolio
VaR_ptf, ES_ptf = var_cvar_series(LOSS_PTF, ALPHA, None)
w = ewma_weights(len(LOSS_PTF), EWMA_LAMBDA)
VaR_ptf_E, ES_ptf_E = var_cvar_series(LOSS_PTF, ALPHA, w)

print(f"== CCS Portafolio VaR/ES (α={ALPHA:.2%}, n={HORIZON_N_DAYS} día[s]) ==")
print(f"  Histórico: VaR = {VaR_ptf:,.2f}  |  ES = {ES_ptf:,.2f}")
print(f"  EWMA     : VaR = {VaR_ptf_E:,.2f} |  ES = {ES_ptf_E:,.2f}")

# Por instrumento
print("\n== Standalone por CCS (Histórico) ==")
display(pd.DataFrame({"VaR": LOSS_BY.quantile(ALPHA), "ES": LOSS_BY[LOSS_PTF>=LOSS_PTF.quantile(ALPHA)].mean()}).round(2))

print("== Standalone por CCS (EWMA) ==")
# ES con EWMA (promedio ponderado en cola)
qE, _ = var_cvar_series(LOSS_PTF, ALPHA, w)
maskE = (LOSS_PTF >= qE).values
w_tail = w[maskE]
if w_tail.sum()>0:
    ES_by_E = (LOSS_BY[maskE].T.dot(w_tail)/w_tail.sum())
    display(pd.DataFrame({"VaR_EWMA": LOSS_BY.apply(lambda s: var_cvar_series(s, ALPHA, w)[0]),
                          "ES_EWMA": ES_by_E}).round(2))
else:
    display(pd.DataFrame({"VaR_EWMA": np.nan, "ES_EWMA": np.nan}, index=LOSS_BY.columns))


== CCS Portafolio VaR/ES (α=99.00%, n=1 día[s]) ==
  Histórico: VaR = 4,487,210.80  |  ES = 5,960,342.99
  EWMA     : VaR = 5,089,460.43 |  ES = 5,379,918.53

== Standalone por CCS (Histórico) ==


,VaR,ES
CCS_1,4487210.8,5960342.99


== Standalone por CCS (EWMA) ==


,VaR_EWMA,ES_EWMA
CCS_1,5089460.43,5379918.53
